In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from combine import combine
from fsUtils import moveFile
from ioUtils import getFile
from timeUtils import clock, elapsed
from pandas import DataFrame, Series, read_csv


import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-11-01 10:32:16.116339


In [2]:
oscarsURL = "https://www.imdb.com/oscars/nominations/?ref_=ev_all"
baftasURL = "https://www.imdb.com/awards-central/baftas/?ref_=ev_all"
goldenglobesURL = "https://www.imdb.com/golden-globes/nominations/?ref_=ev_all"
sagURL = "https://www.imdb.com/event/ev0000598/2019/1"


In [6]:
from webUtils import getWebData

url      = oscarsURL
savename = "awards/IMDB_oscars_base.p"
#getWebData(base=url, savename=savename, useSafari=False, debug=True)

url      = baftasURL
savename = "awards/IMDB_baftas_base.p"
getWebData(base=url, savename=savename, useSafari=False, debug=True)

url      = goldenglobesURL
savename = "awards/IMDB_goldenglobes_base.p"
getWebData(base=url, savename=savename, useSafari=False, debug=True)

url      = sagURL
savename = "awards/IMDB_sag_base.p"
getWebData(base=url, savename=savename, useSafari=False, debug=True)

  --> This file is 61.3kB.
awards/IMDB_baftas_base.p size -> 61 kB
  --> This file is 43.0kB.
awards/IMDB_goldenglobes_base.p size -> 43 kB
  --> This file is 78.3kB.
awards/IMDB_sag_base.p size -> 78 kB


True

# Parse Data

In [60]:
oscarsData

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="app-id=342792525, app-argument=imdb:///?src=mdot" name="apple-itunes-app"/>
<style>
                body#styleguide-v2 {
                    background: url("https://m.media-amazon.com/images/M/MV5BMTg4OTU4Mjk5Nl5BMl5BanBnXkFtZTgwNTI0NTgxNzM@._V1_.jpg") no-repeat fixed center top #fff;
                }
            </style>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>Oscars Guide - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<

In [58]:
from webUtils import getHTML

oscarsData = getHTML("awards/IMDB_oscars_base.p")

In [98]:
from time import sleep
import json
from searchUtils import findPatternExt

def getPageData(data, awardsName):
    dd = data["nomineesWidgetModel"]['eventEditionSummary']
    awards = dd['awards']
    year   = dd['year']
    
    names    = {"baftas": "BAFTA Film Award", "oscars": "Oscars", "sag": "Actor"}
    dataname = names.get(awardsName)
    
    awardsData = {}
    for award in awards:
        print(award['awardName'])
        if award['awardName'] != dataname:
            continue
        for category in award['categories']:
            name     = category["categoryName"]
            nominees = category["nominations"]
            categoryData = {"Winner": [], "Nominees": []}
            for nominee in nominees:
                primary   = nominee['primaryNominees']
                secondary = nominee['secondaryNominees']
                isWinner  = nominee['isWinner']

                primaryData   = [x['name'] for x in primary]
                secondaryData = [x['name'] for x in secondary]
                catData = {"Primary": primaryData, "Secondary": secondaryData}
                if isWinner is True:
                    categoryData["Winner"].append(catData)
                else:
                    categoryData["Nominees"].append(catData)

            awardsData[name] = categoryData
            
    retval = {year: awardsData}
    return retval

def parseOscarsData(awardsName="oscars"):

    awardsData = {}
    files = findPatternExt("awards", pattern="IMDB_{0}_".format(awardsName), ext=".p")
    #print("Found {0} files".format(len(files)))
    for oscarsFile in files:
        if "base.p" in oscarsFile:
            continue
            
        oscarsData = getHTML(oscarsFile)
        for sdata in oscarsData.findAll("script", {"type": "text/javascript"}):

            txt = sdata.text
            pos = txt.find("IMDbReactWidgets.NomineesWidget.push")
            if pos == -1:
                continue
            start = pos + len("IMDbReactWidgets.NomineesWidget.push(")
            end   = txt.rfind("]);") + 1

            txtdata = txt[start:end]
            try:
                data = json.loads(txtdata[len("['center-8-react',"):-1])
            except:
                raise ValueError("Cannot find dictionary in data!!!")

            retval = getPageData(data, awardsName)
            awardsData.update(retval)
        print(len(awardsData))
    return awardsData
    
    
def getOscarsData(awardsName="oscars"):
    awardsData = {}
    
    oscarsData = getHTML("awards/IMDB_{0}_base.p".format(awardsName))
    for sdata in oscarsData.findAll("script", {"type": "text/javascript"}):

        txt = sdata.text
        pos = txt.find("IMDbReactWidgets.EventHistoryWidget.push")
        if pos == -1:
            continue
        start = pos + len("IMDbReactWidgets.EventHistoryWidget.push(")
        end   = txt.rfind("]);") + 1

        txtdata = txt[start:end]
        try:
            data = json.loads(txtdata[len("['right-1-react',"):-1])
        except:
            raise ValueError("Cannot find dictionary in data!!!")

        eventID = data['eventHistoryWidgetModel']['eventId']
        for event in data['eventHistoryWidgetModel']['eventEditions']:
            #eventID  = event['eventEditionId']
            year     = event['year']

            #url      = "https://www.imdb.com/event/ev0000003/2018/1/
            url      = "https://www.imdb.com/event/{0}/{1}/1/".format(eventID, year)
            savename = "awards/IMDB_{0}_{1}.p".format(awardsName, year)
            try:
                getWebData(base=url, savename=savename, useSafari=False, debug=True)        
                sleep(1.5)
            except:
                break
    
    #retval = getPageData(data)
    #awardsData.update(retval)
    #print(len(awardsData))
    
    print("\n\n\n")

In [80]:
#getOscarsData("goldenglobes")
#parseOscarsData()

In [99]:
parseOscarsData("sag")

Actor
Life Achievement Award
1
Life Achievement Award
2
Life Achievement Award
3
Actor
Life Achievement Award
4
Life Achievement Award
5
Life Achievement Award
6
Life Achievement Award
7
Actor
Life Achievement Award
8
Life Achievement Award
9
Life Achievement Award
10
Actor
Life Achievement Award
11
Actor
Life Achievement Award
12
Life Achievement Award
13
Ralph Morgan Award
Life Achievement Award
14
Actor
Life Achievement Award
15
Life Achievement Award
16
Actor
Ralph Morgan Award
Life Achievement Award
17
Life Achievement Award
18
Life Achievement Award
19
Life Achievement Award
20
Actor
Life Achievement Award
21
Life Achievement Award
22
Life Achievement Award
23
Life Achievement Award
24
Actor
Life Achievement Award
25
Actor
Ralph Morgan Award
Life Achievement Award
26
Actor
Life Achievement Award
27
Life Achievement Award
28
Actor
Life Achievement Award
29
Life Achievement Award
30
Actor
Ralph Morgan Award
Life Achievement Award
31
Life Achievement Award
32
Actor
Life Achievement 

{2012: {'Outstanding Performance by a Female Actor in a Drama Series': {'Winner': [{'Primary': ['Jessica Lange'],
     'Secondary': ['American Horror Story']}],
   'Nominees': [{'Primary': ['Glenn Close'], 'Secondary': ['Damages']},
    {'Primary': ['Julianna Margulies'], 'Secondary': ['The Good Wife']},
    {'Primary': ['Kathy Bates'], 'Secondary': ["Harry's Law"]},
    {'Primary': ['Kyra Sedgwick'], 'Secondary': ['The Closer']}]},
  'Outstanding Performance by a Male Actor in a Drama Series': {'Winner': [{'Primary': ['Steve Buscemi'],
     'Secondary': ['Boardwalk Empire']}],
   'Nominees': [{'Primary': ['Bryan Cranston'], 'Secondary': ['Breaking Bad']},
    {'Primary': ['Kyle Chandler'], 'Secondary': ['Friday Night Lights']},
    {'Primary': ['Michael C. Hall'], 'Secondary': ['Dexter']},
    {'Primary': ['Patrick J. Adams'], 'Secondary': ['Suits']}]},
  'Outstanding Performance by a Female Actor in a Comedy Series': {'Winner': [{'Primary': ['Betty White'],
     'Secondary': ['Hot in